### Iba1 4G8 Segmentation and Localisation Script
Created: 18/11/2022
Creator: Emma Davis

#### Workflow
1. Read in 4G8 and Iba1 channels from same area of LCM.
2. Segment microglia for Iba1 channel image and get coords.
3. Segment plaques for 4G8 channel image and get coords.

#### Workflow A
4. For each microglia take the middle coord and find circle areas
    with radius 20um and 50um.
5. Check if plaque area overlaps with either circle areas, define
    microglia with overlap in 20um as plaque niche microglia,
    define microglia with no overlap in 50um as plaque far microglia.

#### Workflow B 1
4. Dilate plaque area to expand out 20um from current plaque border,
    then 50um from current plaque border.
5. For each microglia, check if any of its area lies in the plaque area.
    Microglia with >30% area overlapping 20um ring are plaque niche,
    microglia with <30% area overlapping 50um ring are far
    
#### Workflow B 2
4. Dilate plaque area to expand out 20um from current plaque border,
    then 50um from current plaque border.
5. For each microglia, check if any of its area lies in the plaque area.
    Any area within 20um ring of plaques should be cut as plaque niche,
    any area outside 50um ring of plaques should be cut as plaque far.
</br>
</br>

#### Changelog

02-02-2023: Added DAPI segmentation to more successfully define microglia in
    preparation for morphological analysis. DAPI seg is using stardist.



In [ ]:
%matplotlib inline 

# IMPORTS
import cv2
from datetime import date, datetime
import json
from IPython.display import Image, display
from math import ceil, pi
from matplotlib import pyplot as plt
import multiprocessing
import numpy as np
import os
import pandas as pd
from PIL import ImageOps, Image
from PIL.TiffTags import TAGS
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
import xmltodict
import time
import stardist
from stardist.models import StarDist2D
from stardist.plot import render_label
from csbdeep.utils import normalize


In [ ]:
# SET WORKING DIR
working_dir = "/home/my_project/"
os.chdir(working_dir)
# IMPORT PRETRAINED MODEL FOR MICROGLIA SEGMENTATION
model = keras.models.load_model('microglia_seg_model_0423.h5')


In [ ]:
# GO TO DIRECTORY HOLDING ALL IMAGES TO SEGMENT
# TESTING IBA1 DAPI SEG
lcm_dir = "/training_data/orig/microglia/"

iba1_imgs = sorted([file for file in os.listdir(lcm_dir) if file.endswith('.tif')])


In [ ]:
# ~~~~~~~~~~ ALL FUNCTIONS ~~~~~~~~~~

# CLASS TO BATCH IMAGES INTO NUMPY ARRAYS
class ArrayHelper(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, 2)

            # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
            y[j] -= 1
        return x, y

# FUNCTION TO PERFORM GAMMA CORRECTION ON LCM IMAGES
def gammaCorrection(src, gamma):
    invGamma = 1 / gamma

    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)

    return cv2.LUT(src, table)

# CROP IMAGE INTO 1920x1920 NO OVERLAP
def crop(img_loc, path, height, width, k, page, area):
    im = Image.open(img_loc)
    imgwidth, imgheight = im.size
    img_lst = []
    for i in range(0,imgheight,height):
        for j in range(0,imgwidth,width):
            box = (j, i, j+width, i+height)
            a = im.crop(box)
            #display(a)
            img_lst.append(a)
            k +=1
    return img_lst

# CROP IMAGE INTO 1920x1920 WITH HALF OVERLAP
def crop_ovrlp(img_loc, height, width, k, page, area):
    im = Image.open(img_loc)
    imgwidth, imgheight = im.size
    img_lst = []
    coord_lst = {}
    counter = 0
    for i in range(0,imgheight,ceil(height/2)):
        for j in range(0,imgwidth,ceil(width/2)):
            box = (j, i, j+width, i+height)
            coord_lst[counter] = box
            counter += 1
            a = im.crop(box)
            #display(a)
            img_lst.append(a)
            k +=1
    return img_lst, coord_lst

# DISPLAY PREDICTED TRIMAP
def display_mask(i):
    mask = np.argmax(val_preds[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
    display(img)

# CONVERT PREDICTED MASK TO BINARY
def mask_to_binary(i):
    mask = np.argmax(i, axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
    img = np.asarray(img)
    proc_img = img.copy()
    if np.all(proc_img==0):
        img = Image.fromarray(proc_img)
    else:
        proc_img[proc_img == 0] = 255
        proc_img[proc_img == 127] = 0
        img = Image.fromarray(proc_img)
    
    if np.all(proc_img==255):
        proc_img[proc_img == 255] = 0
        img = Image.fromarray(proc_img)
    
    #display(img)
    return img

# FUNCTION TO ALLOW CHUNKING ON 5 COORDS AT A TIME
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# FUNCTION TO EXPORT CONTOURS AS COORDS
def contour_to_coords(final_contours, img_name, dir_path, dir_name, pxl_to_lcm):
    total_area = 0
    
    # OPEN TIF IMAGES AND GET METADATA
    img = Image.open(dir_path + "/Iba1/" + img_name + ".tif")
    meta_dict = {TAGS[key] : img.tag[key] for key in img.tag_v2}

    # PARSE METADATA AS DICT
    meta_dict['ImageDescription'] = str(meta_dict['ImageDescription']).replace(
        '\\x00', '').replace("b\'", '').replace("\\r\\n\'", '')
    meta_dict = xmltodict.parse(meta_dict['ImageDescription'])
    
    # DRAW CONTOURS WITH AREA LARGER THAN x, ALSO ADD TO LIST OF THRESHOLDED CONTOURS
    # FOR ALL COORDINATES, ADD X AND Y OFFSET FROM METADATA TO MATCH LCM
    x_offset = int(meta_dict['RecordList']['PictureInfo']['Picture']['StagePosition'][0]['#text'])
    y_offset = int(meta_dict['RecordList']['PictureInfo']['Picture']['StagePosition'][1]['#text'])
    
    #final_contours = final_contours.astype(float)
    
    # PIX TO LCM
    if pxl_to_lcm == 1:
        for coord in final_contours:
            coord[0][0] = float(coord[0][0]) /5.703
            coord[0][1] = float(coord[0][1]) /5.703
            coord[0][0] = float(coord[0][0]) + x_offset - 360
            coord[0][1] = float(coord[0][1]) + y_offset - 397.7
    
    # EXPORT COORDS FOR LCM, AVERAGE AREA FOR ONE CELL IS 5000-6000
    date = datetime.today()
    date = date.strftime("%d.%m.%Y")
    time = datetime.now()
    time = time.strftime("%H:%M:%S")

    header = str("PALMRobo Elements\nVersion:\tV 4.9.0.0\nDate, Time:\t" + str(date) + "\t" +
                 str(time) + "\n\nMICROMETER\nElements :\n\nType\tColor\tThickness\tNo\tLaser function" +
                 "\tCutShot\tArea\tZ\tWell\tObjective\tComment\tCoordinates\t\n")

    for i in range(len(final_contours)):
        number_roi = i
        area_roi = cv2.contourArea(final_contours[i])
        total_area += area_roi
        roi_str = str("\nFreehand\tgreen\t2\t" + str(number_roi) + "\tCut\t0,0\t" + str(area_roi) +
                  "\t6910\tmanual\t40x - LD Plan-Neofluar 40x/0.6 Korr M27\t.\t")
        header = header + roi_str

        # 5 COORDS MAX ON EACH LINE
        for j in chunker(range(len(final_contours[i])), 5):
            coords = final_contours[i]
            line = "."
            for k in j:
                coord = coords[k]
                line = line + "\t" + str(coord[0][0]) + "," + str(coord[0][1])
            header = header + "\n" + line

    # WRITE ELEMENTS
    elems = open(dir_path + dir_name + img_name + "_elems.txt", "w")
    n = elems.write(header)
    elems.close()

    print("Total area of ROIs: " + str(total_area))
    
    return total_area

# CONVERT PIXEL CONTOURS TO LCM CONTOUR COORDINATES
def pxl_to_lcm(contour, x_offset, y_offset):
    contour_lcm = contour.copy()
    for coord in contour_lcm:
        coord[0][0] = float(coord[0][0]) /5.703
        coord[0][1] = float(coord[0][1]) /5.703
        coord[0][0] = float(coord[0][0]) + x_offset - 360
        coord[0][1] = float(coord[0][1]) + y_offset - 397.7
    
    return contour_lcm

# MICROGLIA SEGMENTATION
def microglia_segmentation(img_name, dir_path, denoise=2, gamma=0.9, dilate=15, erode=3):
    # OPEN TIF IMAGES AND GET METADATA
    img = Image.open(dir_path + img_name + ".tif")
    meta_dict = {TAGS[key] : img.tag[key] for key in img.tag_v2}

    # PARSE METADATA AS DICT
    meta_dict['ImageDescription'] = str(meta_dict['ImageDescription']).replace(
        '\\x00', '').replace("b\'", '').replace("\\r\\n\'", '')
    meta_dict = xmltodict.parse(meta_dict['ImageDescription'])

    # DENOISE IMAGE
    img = np.array(img)
    img = cv2.fastNlMeansDenoising(img, h=denoise)#h=16

    # ADJUST EXPOSURE/GAMMA
    img = gammaCorrection(img, gamma=gamma)

    # ADJUST CONTRAST VIA THRESHOLDING
    #ret,img = cv2.threshold(img, 30, 255, cv2.THRESH_TOZERO)

    # INCREASE CONTRAST
    # converting to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)

    # Applying CLAHE to L-channel
    # feel free to try different values for the limit and grid size:
    clahe = cv2.createCLAHE(clipLimit=1, tileGridSize=(8,8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))

    # Converting image from LAB Color model to BGR color spcae
    img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # DISPLAY IMPROVED IMAGE
    plt.imshow(img, interpolation='nearest')
    plt.show()
    
    # SAVE CORRECTED IMAGE
    img = Image.fromarray(img)

    # CROP OUT BAR AT BOTTOM
    img = img.crop((0, 0, 1936, 1400))
    
    # CHECK IF WHOLE CLEANED DIRECTORY EXISTS, IF NOT MAKE ONE AND SAVE IMG
    if not os.path.exists(dir_path + "/Iba1_whole_clean"):
        os.makedirs(dir_path + "/Iba1_whole_clean")

    img.save(dir_path + "/Iba1_whole_clean/" + img_name + ".png")

    # CROP WHOLE IMAGE IN SLIDING WINDOW FASHION
    img_lst, coord_lst = crop_ovrlp(dir_path + "/Iba1_whole_clean/" + img_name + ".png",
                                    384, 384, 0, 1, 417316)
    
    # CHECK IF WHOLE CLEANED DIRECTORY EXISTS, IF NOT MAKE ONE AND SAVE IMG
    if not os.path.exists(dir_path + "/Iba1_crops/" + img_name):
        os.makedirs(dir_path + "/Iba1_crops/" + img_name)
    
    # SAVE CROPPED IMAGES
    for i in range(len(img_lst)):
        img_lst[i].save(dir_path + "/Iba1_crops/" + img_name + "/" + str(i).zfill(3) + '.png')

    # SAVE CROPPED COORDS AS TXT FILE
    with open(dir_path + "/Iba1_crops/" + img_name + "/coords.txt", 'w') as file:
        file.write(json.dumps(coord_lst))
        
    # SPLIT IMAGE INTO MANAGABLE CHUNKS FOR MODEL WHILE RETAINING COORDINATE SYSTEM IN AN ARRAY SOMEWHERE
    input_dir = dir_path + "/Iba1_crops/" + img_name + "/"
    target_dir = "/training_data/augmented/masks/"

    img_size = (384, 384)
    num_classes = 3
    batch_size = 88

    input_img_paths = sorted(
        [
            os.path.join(input_dir, fname)
            for fname in os.listdir(input_dir)
            if fname.endswith(".png")
        ]
    )
    target_img_paths = sorted(
        [
            os.path.join(target_dir, fname)
            for fname in os.listdir(target_dir)
            if fname.endswith(".png") and not fname.startswith(".")
        ]
    )

    print("Number of samples:", len(input_img_paths))

    for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
        print(input_path, "|", target_path)

    pred_gen_parallel = []
    for x in range(int(batch_size/8)):
        pred_gen_parallel.append(ArrayHelper(8, img_size, input_img_paths[x*8:(x*8)+8],
                                         target_img_paths[x*8:(x*8)+8]))

    # TIME PREDICTION
    start = time.time()

    # RUN PREDICTION IN CHUNKED BATCHES FOR SPEED
    val_preds = []
    for pred_gen in pred_gen_parallel:
        val_preds.extend(model.predict(pred_gen))

    end = time.time()
    print("Prediction took " + str(end - start) + " seconds.")
    
    # CHECK IF WHOLE CLEANED DIRECTORY EXISTS, IF NOT MAKE ONE AND SAVE IMG
    if not os.path.exists(dir_path + "/Iba1_crops_pred/" + img_name):
        os.makedirs(dir_path + "/Iba1_crops_pred/" + img_name)
        
    # MAKE DIRECTORY FOR COORDS
    if not os.path.exists(dir_path + "/Iba1_coords/"):
        os.makedirs(dir_path + "/Iba1_coords/")

    # GET PREDICTED MASKS FOR ALL CROPS AND CONVERT TO BINARY BEFORE SAVING
    for i in range(batch_size):
        bin_mask = mask_to_binary(val_preds[i])
        bin_mask.save(dir_path + "/Iba1_crops_pred/" + img_name + "/" + str(i).zfill(3) + '.png')

    # STITCH UP CROPPED PREDICTED MASKS TO GET WHOLE PREDICTED MASK FOR IMAGE
    mask = np.zeros((1728, 2304))

    # GET COORDS BACK
    with open(dir_path + "/Iba1_crops/" + img_name + "/coords.txt") as f:
        coord_dict = f.read()
    
    coord_dict = json.loads(coord_dict)

    for sub_mask, bbox in zip(val_preds, coord_dict.values()):
        x1, y1, x2, y2 = bbox
        sub_mask = np.asarray(mask_to_binary(sub_mask))
        mask[y1:y2, x1:x2] += sub_mask

    mask = ((mask>=1)*255.0).astype(np.uint8)
    
    # DILATE AND ERODE MASK
    kernel = np.ones((dilate, dilate), np.uint8)
    mask = cv2.dilate(mask, kernel)

    kernel = np.ones((erode, erode), np.uint8)
    mask = cv2.erode(mask, kernel)

    mask = Image.fromarray(mask)

    # CROP TO SAME SIZE AS ORIGINAL
    mask = mask.crop((0, 0, 1936, 1400))

    # NEED TO ROTATE 180 TO MAKE COORDS MATCH LCM
    mask = mask.transpose(Image.ROTATE_180)
    
    # GET NUMERIC ROI COORDINATES FROM PREDICTIVE MASKS (OPENCV FIND CONTOURS ON BINARY MASK?
    np_mask = np.array(mask)
    contours, hierarchy = cv2.findContours(np_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    orig_img = np.array(
        Image.open(dir_path + "/Iba1_whole_clean/" + img_name + '.png'))

    # ROTATE ORIGINAL IMAGE TO MATCH LCM COORD SYSTEM
    orig_img = Image.fromarray(orig_img)
    orig_img = orig_img.transpose(Image.ROTATE_180)
    orig_img = np.asarray(orig_img)

    # DRAW CONTOURS WITH AREA LARGER THAN x, ALSO ADD TO LIST OF THRESHOLDED CONTOURS
    # FOR ALL COORDINATES, ADD X AND Y OFFSET FROM METADATA TO MATCH LCM
    x_offset = int(meta_dict['RecordList']['PictureInfo']['Picture']['StagePosition'][0]['#text'])
    y_offset = int(meta_dict['RecordList']['PictureInfo']['Picture']['StagePosition'][1]['#text'])

    final_contours_lcm = []
    final_contours_pxl = []
    for contour in contours:
        if cv2.contourArea(contour) > 10000:#> 3000:
            # ADDED CHECK TO SEE IF MICROGLIA HAS DAPI
            #dapi_check = microglia_dapi_count(dapi_dict[str(img_name + ".tif")], contour)
            dapi_check = True
            if dapi_check:
                cv2.drawContours(orig_img, contour, -1, (0, 255, 0), 3)
                final_contours_pxl.append(contour.copy())

                for coord in contour:
                    coord[0][0] /= 5.703
                    coord[0][1] /= 5.703
                    coord[0][0] += x_offset - 360
                    coord[0][1] += y_offset - 397.7
                final_contours_lcm.append(contour)

    plt.imshow(orig_img,cmap='gray')

    # SAVE IMAGE WITH ROIS DRAWN
    orig_img = Image.fromarray(orig_img)
    orig_img.save(dir_path + "/Iba1_whole_clean/" + img_name + "_predROIs.png")
    
    area = contour_to_coords(final_contours_lcm, img_name, dir_path, "/Iba1_coords/", 0)
    return area, final_contours_lcm, final_contours_pxl



In [ ]:
# RUN IBA1 SEGMENTATION
iba_rois_lcm = {}
iba_rois_pxl = {}
for iba1 in iba1_imgs:
    # GET CONTOURS AND ADD TO DICT
    area, contours_lcm, contours_pxl = microglia_segmentation(iba1.replace(".tif", ""), lcm_dir)
